# **Group 19 Project**  **GIVE BETTER TITLE**

# **Introduction**

Video games have evolved from simple pastimes into complex environments that offer rich data about user behavior and interaction. This report is grounded in a real-world data science project conducted by a research group in Computer Science at UBC, led by Frank Wood. The team has established a Minecraft server where every action taken by players is recorded. By capturing this data, the researchers aim to unlock insights into how individuals navigate and interact within virtual worlds.

The project has multiple objectives, and we will focus on understanding the characteristics and behaviors that most predict a player's likelihood to subscribe to a game-related newsletter. This targeted approach helps ensure that sufficient resources—such as software licenses and server hardware—are available to support the anticipated influx of players. By investigating player behavior through detailed analytics, the study aims to inform future strategies for engagement, recruitment, and resource allocation in online gaming communities. This report will detail the specific methodologies used to analyze the player data, the key findings related to newsletter subscription behavior, and the implications of these findings.

# **Question**

Are age and gender predictive of subscription status to a game related newsletter in the player.csv data set?

## Data Set Description

There a two datasets containing information on players on the MineCraft server; "players.csv" and "sessions.csv". 

The "players.csv" dataset contains observations collected for multiple different variables from people who played on the MineCraft server. The data frame contains 7 variables and 196 rows of data, producing 1372 observations in total. The variables are ordered in the table left to right are:

- `Experience`
    - This variable describes the level at which each player is at in terms of playing the game.
    - This variable is represented by a string value that can be either Amateur, Beginner, Regular, Pro, or Veteran
- `Subscribe`
    - This variable describes whether or not the player is subscribed to a game-related newsletter.
    - This variable is represented by a boolean value (either True or False) 
- `Hashed Email`
    - This variable describes lists each players email in a hashed format. 
    - This variable is represented by a string  
- `Hours Played`
    - This variable describeshow many hours each player spent playing the game (in hours). 
    - This variable is represented by a float value (number with a decimal value)
- `Name`
    - This variable states the players first name
    - This variable is represented by a string  
- `Gender`
    - This variable describes the gender of each player. 
    - This variable is represented by a string value that can be either Agender, Female, Male, Non-binary, other, Prefer not to say, or Two-Spirited
- `Age`
    - This variable describes the age of the players (in years) 
    - This variable represented by an integer value (whole number) 

**This is the data set that will be used in the analysis.**

The "sessions.csv" data contains observations collected for multiple different variables from people who played on the MineCraft server. The data frame contains 5 variables and 1535 rows of data, producing 7675 observations in total. The variables are ordered in the table from left to right are:

- `hashedEmail`
    - This variable gives a string of letters and numbers that represent the players email address. 
    - This variable is represented by a string  
- `start_time`
    - This variable gives the exact date (DD/MM/YR) and time (24 hour clock) that the player started their session.
    -  This variable is represented by a string
- `end_time`
    - This variable the exact date (DD/MM/YR) and time (24 hour clock) that the player ended their session. 
    - This variable is represented by a string  
- `original_start_time`
    - This variable describes the original start time of players  (**IDK HOW TO DESCRIBE IT BETTER**)
    - This variable is represented by a float value (number with a decimal value)
- `original_end_time`
    - This variable describes the original end time of players (**SAME HERE**)
    - This variable is represented by a float value (number with a decimal value)


This data set will not be used in the analysis. 

In [ ]:
library(tidyverse)
library(rvest)
library(dplyr)
library(tidymodels)

In [ ]:
# url_sessions <- "https://raw.githubusercontent.com/IFQXK/DSCI-100-project-group-19/refs/heads/main/sessions.csv"
# sessions_data <- read.csv(url_sessions)
# head(sessions_data)

url_players <- "https://raw.githubusercontent.com/IFQXK/DSCI-100-project-group-19/refs/heads/main/players.csv"
players_data <- read.csv(url_players)
head(players_data)

In [ ]:
players_data_raw <- players_data |>
    mutate(subscribe = as.factor(subscribe)) |>
    select(subscribe, played_hours, Age)

players_data_fixed <- filter(players_data_raw, is.na = TRUE)

head(players_data_fixed)

In [ ]:
players_split <- initial_split(players_data_fixed, prop = 0.75, strata = subscribe)
players_training <- training(players_split)
players_testing <- testing(players_split)

This is what the first 6 lines of code looks like without wrangling

In [ ]:
mean_table <- players_training |>
summarize(
Average_Age = mean(Age, na.rm = TRUE),)

mean_table


## Statistics Summary of Variables used in Specific Question

`Age` :

Mean: 20.9

Max: 50

Min: 8

`Gender` :

Agender: 2

Female: 26

Male: 93

Non-binary: 13

Other: 1

Perfer not to say: 7

Two_Spirited: 5

`Subscribe`

True: 108

False: 39

Above is the summary for each statistic used in the analysis. If an integer value, mean, max, and min were calculated. If the stat was a character value, the count was summarized per category. 

In [ ]:
# Scatter graph comparing age and experience
options(repr.plot.width = 10, repr.plot.height = 4)
gender_bar <- players_training |>
            ggplot(aes(x = played_hours, fill = subscribe)) +
            geom_histogram(position = "dodge") +
            labs(x = "Gender", title = "Relationship between gender and subscription status") + 
            theme(text = element_text(size = 14))       
gender_bar

In [ ]:
age_histogram <- players_training |>
            ggplot(aes(x = Age, fill = subscribe)) +
            geom_histogram(position = "dodge", binwidth = 5) +
            labs(x = "Age (in years)", title = "Relationship between age and subscription status") + 
            theme(text = element_text(size = 14))       
age_histogram

^ creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis. **idk if this counts**

-**I think this represents the total age which I dont think is what we want**

# Aanlysis

In [ ]:
scatter_plot <- players_training |>
            ggplot(aes(x = Age, y = played_hours, colour = subscribe)) +
            geom_point() +
            labs(y = "Gender", x = "Age (in years)", title = "Relationship between age and experience") + 
            theme(text = element_text(size = 14))
    
scatter_plot

In [ ]:
players_recipe <- recipe(subscribe ~ Age + played_hours, data = players_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

players_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
    set_engine("kknn") |>
    set_mode("classification")

players_workflow <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(players_spec) |>
    fit(data = players_training)
players_workflow

In [ ]:
players_test_prediction <- predict(players_workflow, players_testing) |>
    bind_cols(players_testing)

players_test_accuracy <- players_test_prediction |>
    metrics(truth = subscribe, estimate = .pred_class)
players_test_accuracy

In [ ]:
players_mat <- players_test_prediction |> 
      conf_mat(truth = subscribe, estimate = .pred_class)

players_mat

In [ ]:
players_vfold <- vfold_cv(players_testing, v = 5, strata = subscribe)

players_resample_fit <- workflow() |>
      add_recipe(players_recipe) |>
      add_model(players_spec) |>
      fit_resamples(resamples = players_vfold) |>
    collect_metrics()

players_resample_fit

In [ ]:
gridvals <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

players_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

players_results <- workflow() |>
      add_recipe(players_recipe) |>
      add_model(players_tune) |>
      tune_grid(resamples = players_vfold, grid = gridvals) |>
      collect_metrics()

players_results

In [ ]:
accuracies <- players_results |> 
      filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
      geom_point() +
      geom_line() +
      labs(x = "Neighbors", y = "Accuracy Estimate") +
      scale_x_continuous(breaks = seq(0, 14, by = 1)) +  # adjusting the x-axis
      scale_y_continuous(limits = c(0.4, 1.0)) # adjusting the y-axis

accuracy_versus_k


From the plot above, we can see that $K = 2$ provides the highest accuracy. Larger $K$ values result in a reduced accuracy estimate.